<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/03_wohnungen_verkauf/wohnungen_verkauf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recherche von Anzeigen für den Verkauf von Wohnungen

Wir werden das Archiv der Anzeigen für den Verkauf von Wohnungen in St. Petersburg und benachbarten Siedlungen über mehrere Jahre analysieren. Ziel ist es zu lernen, wie man den Marktwert von Immobilien ermittelt. Es ist notwendig, die Parameter festzulegen, die den Aufbau eines automatisierten Systems zur Verfolgung von Anomalien und betrügerischen Aktivitäten ermöglichen.

Für jede zum Verkauf stehende Wohnung stehen zwei Arten von Daten zur Verfügung. Die ersten werden vom Benutzer eingegeben, die zweiten werden automatisch auf der Grundlage kartografischer Daten erhalten. Zum Beispiel die Entfernung zum Zentrum, Flughafen, nächsten Park und Stausee.

# Datenexploration

In [17]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd

In [18]:
# Wir laden Tabellen mit Daten
data = pd.read_csv('/content/real_estate_data.csv', sep='\t')

In [19]:
# Wir werden die erhaltenen Daten studieren
display(data.info())
display(data.shape)
display(data.head())
display(data.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

(23699, 22)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
23694,9,9700000.0,133.81,2017-03-21T00:00:00,3,3.7,5.0,73.3,3,NaN,...,13.83,NaN,Санкт-Петербург,24665.0,4232.0,1.0,796.0,3.0,381.0,NaN
23695,14,3100000.0,59.00,2018-01-15T00:00:00,3,NaN,5.0,38.0,4,NaN,...,8.50,NaN,Тосно,NaN,NaN,NaN,NaN,NaN,NaN,45.0
23696,18,2500000.0,56.70,2018-02-11T00:00:00,2,NaN,3.0,29.7,1,NaN,...,NaN,NaN,село Рождествено,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23697,13,11475000.0,76.75,2017-03-28T00:00:00,2,3.0,17.0,NaN,12,NaN,...,23.30,2.0,Санкт-Петербург,39140.0,10364.0,2.0,173.0,3.0,196.0,602.0
23698,4,1350000.0,32.30,2017-07-21T00:00:00,1,2.5,5.0,12.3,1,NaN,...,9.00,NaN,поселок Новый Учхоз,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Schlussfolgerungen**

Nachdem wir die Daten untersucht haben, sehen wir, dass der Datenrahmen aus 23699 Zeilen und 22 Spalten besteht. Die Tabelle ist sehr unhandlich, einige Spalten werden zur Beantwortung der Fragen nicht benötigt. Separate Spalten enthalten Nullwerte, irgendwo mehr als die Hälfte davon. Die first_day_exposition-Spalte enthält Informationen über Zeit und Datum, hat aber das Objektdatenformat, die Zeit ist überall Null, sie wird nicht für die Recherche benötigt.

# Datenvorverarbeitung

In [20]:
# Wir werden Spalten entfernen, die für die Datenanalyse nicht benötigt werden
data = data.drop(['total_images', 'is_apartment', 'studio', 'open_plan', 'airports_nearest', 'parks_around3000', 'parks_nearest', 'ponds_around3000', 'ponds_nearest'], axis=1)

In [21]:
# Wir zählen die Anzahl der Duplikate mit einer Reihe von Methoden data.duplicated().sum()
display(data.duplicated().sum())

0

In [22]:
# Wir entfernen die Zeilen, in denen es keine Informationen über die Anzahl der Stockwerke im Haus gibt.
# Es gibt nicht viele solcher Zeilen, das Löschen wird das Ergebnis nicht wesentlich beeinflussen.
data.dropna(subset = ['floors_total'], inplace = True)

In [23]:
# Wir füllen die Nullwerte in der Balkonspalte mit 0-Werten, da es wahrscheinlich keine Balkone gibt.
data['balcony'] = data['balcony'].fillna(0)

In [24]:
# Wir werden die Nullwerte in der Spalte cityCenters_nearest mit dem Medianwert für diese Spalte füllen.
# Wir werden Zeilen mit leeren Werten in dieser Spalte nicht löschen, da zu viele von ihnen.
data['cityCenters_nearest'] = data['cityCenters_nearest'].fillna(data['cityCenters_nearest'].median())

In [25]:
# Wir werden das Datenformat für die Spalte first_day_exposition ändern
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')

In [26]:
# Wir übersetzen die Werte der Spalten last_price, Floors_total, Balcony in ganze Zahlen
data['last_price'] = data['last_price'].astype('int')
data['balcony'] = data['balcony'].astype('int')
data['floors_total'] = data['floors_total'].astype('int')

In [27]:
# Mal sehen was passiert
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23698
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   last_price            23613 non-null  int64         
 1   total_area            23613 non-null  float64       
 2   first_day_exposition  23613 non-null  datetime64[ns]
 3   rooms                 23613 non-null  int64         
 4   ceiling_height        14495 non-null  float64       
 5   floors_total          23613 non-null  int64         
 6   living_area           21743 non-null  float64       
 7   floor                 23613 non-null  int64         
 8   kitchen_area          21382 non-null  float64       
 9   balcony               23613 non-null  int64         
 10  locality_name         23565 non-null  object        
 11  cityCenters_nearest   23613 non-null  float64       
 12  days_exposition       20441 non-null  float64       
dtypes: datetime64[ns

,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,locality_name,cityCenters_nearest,days_exposition
0,13000000,108.0,2019-03-07,3,2.70,16,51.0,8,25.0,0,Санкт-Петербург,16028.0,NaN
1,3350000,40.4,2018-12-04,1,NaN,11,18.6,1,11.0,2,посёлок Шушары,18603.0,81.0
2,5196000,56.0,2015-08-20,2,NaN,5,34.3,4,8.3,0,Санкт-Петербург,13933.0,558.0
3,64900000,159.0,2015-07-24,3,NaN,14,NaN,9,NaN,0,Санкт-Петербург,6800.0,424.0
4,10000000,100.0,2018-06-19,2,3.03,14,32.0,13,41.0,0,Санкт-Петербург,8098.0,121.0


In [28]:
# Wir werden die Nullwerte in die Spalten living_area und kitchen_area je nach Anzahl der Zimmer in der Wohnung eintragen
for rooms in data['rooms'].unique():
    median = data.loc[data['rooms'] == rooms, 'living_area'].median()
    data.loc[(data['rooms'] == rooms) & (data['living_area'].isna()), 'living_area'] = median

    median = data.loc[data['rooms'] == rooms, 'kitchen_area'].median()
    data.loc[(data['rooms'] == rooms) & (data['kitchen_area'].isna()), 'kitchen_area'] = median
    
data.info()
data.head() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23698
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   last_price            23613 non-null  int64         
 1   total_area            23613 non-null  float64       
 2   first_day_exposition  23613 non-null  datetime64[ns]
 3   rooms                 23613 non-null  int64         
 4   ceiling_height        14495 non-null  float64       
 5   floors_total          23613 non-null  int64         
 6   living_area           23613 non-null  float64       
 7   floor                 23613 non-null  int64         
 8   kitchen_area          23419 non-null  float64       
 9   balcony               23613 non-null  int64         
 10  locality_name         23565 non-null  object        
 11  cityCenters_nearest   23613 non-null  float64       
 12  days_exposition       20441 non-null  float64       
dtypes: datetime64[ns

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,locality_name,cityCenters_nearest,days_exposition
0,13000000,108.0,2019-03-07,3,2.70,16,51.0,8,25.0,0,Санкт-Петербург,16028.0,NaN
1,3350000,40.4,2018-12-04,1,NaN,11,18.6,1,11.0,2,посёлок Шушары,18603.0,81.0
2,5196000,56.0,2015-08-20,2,NaN,5,34.3,4,8.3,0,Санкт-Петербург,13933.0,558.0
3,64900000,159.0,2015-07-24,3,NaN,14,45.0,9,9.5,0,Санкт-Петербург,6800.0,424.0
4,10000000,100.0,2018-06-19,2,3.03,14,32.0,13,41.0,0,Санкт-Петербург,8098.0,121.0


**Schlussfolgerungen**

Es gibt keine Duplikate. Spalten mit Daten, die für die Analyse nicht benötigt werden, wurden entfernt, um die Tabelle nicht zu überladen. Wir haben die Nullwerte auf unterschiedliche Weise ausgefüllt, abhängig von der Anzahl der Nullwerte und der Bedeutung der in der Spalte enthaltenen Daten. Separat haben wir Nullwerte in die Spalten eingetragen, die Informationen über Wohnfläche und Küchenbereich enthalten: Wir haben Medianwerte in Abhängigkeit von der Anzahl der Zimmer in der Wohnung angegeben, damit die Berechnungen genauer waren. Zur weiteren Analyse haben wir das Datenformat in der Spalte first_day_exposition in datetime64[ns] geändert. Zur leichteren Wahrnehmung wurden die Werte in den einzelnen Spalten in Ganzzahlen umgewandelt. Die Daten sind nun bereit für die Analyse.

# Berechnungen durchführen